In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import data
data = pd.read_csv("../input/abalone-dataset/abalone.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
# Check for missing values
data.isna().sum()

In [ ]:
unique_sex_values = data['Sex'].unique()
print(unique_sex_values)

In [ ]:
# Map 'Sex' to integers: 'M' -> 0, 'F' -> 1, 'I' -> 2
data['Sex'] = data['Sex'].map({'M': 0, 'F': 1, 'I': 2})

In [ ]:
 data['age'] = data['Rings']+1.5
 data.drop('Rings', axis = 1, inplace = True)

In [ ]:
data.describe()

In [ ]:
#Histogram plotting of all the numerical or qualitative variables present in our dataset.
#Helps us understand if there is normalcy in our data or not, from below we can say that our data seems normal.
plt.style.use('seaborn-whitegrid')
data.hist(bins=20, figsize=(14,10), color='#069EE1')
plt.show();

In [ ]:
# Split the data into features (X) and target (y)
X = data.drop("age", axis=1)
y = data["age"]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Create and train a Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Create and train a Gradient Boosting Regressor model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Create and train a LightGBM Regressor model
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
lgbm_model.fit(X_train, y_train)

# Create and train an XGBoost Regressor model
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data for each model
y_pred_rf = rf_model.predict(X_test)
y_pred_gb = gb_model.predict(X_test)
y_pred_lgbm = lgbm_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)


In [ ]:
# Evaluate each model
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - Mean Squared Error:", mse)
    print(f"{model_name} - R-squared:", r2)

In [ ]:
evaluate_model(y_test, y_pred_rf, "Random Forest")
evaluate_model(y_test, y_pred_gb, "Gradient Boosting")
evaluate_model(y_test, y_pred_lgbm, "LightGBM")
evaluate_model(y_test, y_pred_xgb, "XGBoost")


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Define hyperparameters to search over
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'num_leaves': [31, 63, 127],
    'min_child_samples': [10, 20, 30],
}

# Create the LightGBM Regressor
lgbm_model = lgb.LGBMRegressor(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=lgbm_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_lgbm_model = grid_search.best_estimator_

# Train the model with the best hyperparameters
best_lgbm_model.fit(X_train, y_train)

# Evaluate the model's performance
y_pred_lgbm = best_lgbm_model.predict(X_test)
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
r2_lgbm = r2_score(y_test, y_pred_lgbm)

print("Best Hyperparameters:", best_params)
print("Mean Squared Error (LightGBM):", mse_lgbm)
print("R-squared (LightGBM):", r2_lgbm)


Based on the provided metrics, the LightGBM model has the lowest MSE (indicating slightly better predictive accuracy) and the highest R-squared value (indicating a better fit to the data). Therefore, if predictive accuracy and model fit are the primary criteria, you might consider LightGBM as the best model in this scenario.

In [ ]:
!pip install shap


In [ ]:
!pip install lime

In [ ]:
#Performing shap analysis for above regression.
import shap
shap.initjs()
best_lgbm_model.fit(X_test, y_test)


In [ ]:
import shap

# Initialize the SHAP explainer with your LightGBM model
explainer = shap.TreeExplainer(best_lgbm_model)

# Calculate SHAP values for the entire test set
shap_values = explainer.shap_values(X_test)

# Plot the summary plot for the SHAP values
shap.summary_plot(shap_values, X_test)


In [ ]:
# plot the SHAP values for the 10th observation 

shap.initjs()
shap.force_plot(explainer.expected_value,shap_values[10,:], X_test.iloc[10,:]) 

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, X_test)

In [ ]:
import shap

# Assuming you have a list of feature names in the same order as your shap_values
feature_names = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']

# Then, you can pass the feature_names to shap.summary_plot
shap.summary_plot(shap_values, max_display=15, show=False, plot_type='bar', feature_names=feature_names)


In [ ]:
# Drop the 'Length,' 'Viscera weight,' and 'Sex' columns
data = data.drop(columns=["Length","Diameter", "Height"])

# Split the data into features (X) and target (y)
X = data.drop("age", axis=1)
y = data["age"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


best_lgbm_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_lgbm_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)